<a href="https://colab.research.google.com/github/ayulockin/SwAV-TF/blob/master/linear_evaluation/Linear_Evaluation_10_Epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setups

In [ ]:
!git clone https://github.com/ayulockin/SwAV-TF.git

Cloning into 'SwAV-TF'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 179 (delta 80), reused 26 (delta 11), pack-reused 0
Receiving objects: 100% (179/179), 13.73 MiB | 16.16 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [ ]:
import sys
sys.path.append('SwAV-TF/utils')

import multicrop_dataset
import architecture

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential, Model

import matplotlib.pyplot as plt
import numpy as np
import random
import time
import os

from tqdm import tqdm
from imutils import paths

tf.random.set_seed(666)
np.random.seed(666)

tfds.disable_progress_bar()

#### W&B - Experiment Tracking

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#### Restoring model weights from GCS Bucket - Model trained for 10 epochs

In [ ]:
from tensorflow.keras.utils import get_file

In [ ]:
feature_backbone_urlpath = "https://storage.googleapis.com/swav-tf/feature_backbone_10_epochs.h5"
prototype_urlpath = "https://storage.googleapis.com/swav-tf/projection_prototype_10_epochs.h5"

In [ ]:
feature_backbone_weights = get_file('swav_feature_weights', feature_backbone_urlpath)
prototype_weights = get_file('swav_prototype_projection_weights', prototype_urlpath)

17907712/17900192 [==============================] - 0s 0us/step


#### Dataset gathering and preparation

In [ ]:
# Gather Flowers dataset
train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:85%]", "train[85%:]"],
    as_supervised=True
)

AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 256

@tf.function
def scale_resize_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (224, 224)) # Resizing to highest resolution used while training swav
    return (image, label)

training_ds = (
    train_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

testing_ds = (
    validation_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.



Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


#### Get SwAV architecture and Build Linear Model

In [ ]:
def get_linear_classifier(alpha=1e-6):
    # input placeholder
    inputs = Input(shape=(224, 224, 3))
    # get swav baseline model architecture
    feature_backbone = architecture.get_resnet_backbone()
    # load trained weights
    feature_backbone.load_weights(feature_backbone_weights)
    feature_backbone.trainable = False

    x = feature_backbone(inputs, training=False)
    outputs = Dense(5, activation="softmax", 
                    kernel_regularizer=tf.keras.regularizers.L2(alpha))(x)
    linear_model = Model(inputs, outputs)

    return linear_model

In [ ]:
tf.keras.backend.clear_session()
model = get_linear_classifier()
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


#### Callback

In [ ]:
# Early Stopping to prevent overfitting
early_stopper =a tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, verbose=2, restore_best_weights=True)

# Training Linear Classifier - Without Augmentation

#### Training

In [ ]:
# get model and compile
tf.keras.backend.clear_session()
model = get_linear_classifier(alpha=1e-6)
model.summary()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer="adam")

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train 
history = model.fit(training_ds,
                 validation_data=(testing_ds),
                 epochs=100,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


Epoch 1/100
13/13 [==============================] - 14s 1s/step - loss: 2.2577 - acc: 0.2333 - val_loss: 1.9785 - val_acc: 0.2673
Epoch 2/100
13/13 [==============================] - 12s 960ms/step - loss: 1.7493 - acc: 0.2804 - val_loss: 1.7935 - val_acc: 0.2873
Epoch 3/100
13/13 [==============================] - 12s 957ms/step - loss: 1.6388 - acc: 0.3000 - val_loss: 1.7007 - val_acc: 0.2691
Epoch 4/100
13/13 [==============================] - 12s 930ms/step - loss: 1.5845 - acc: 0.3067 - val_loss: 1.6058 - val_acc: 0.3000
Epoch 5/100
13/13 [==============================] - 12s 911ms/step - loss: 1.5634 - acc: 0.3212 - val_loss: 1.5892 - val_acc: 0.2945
Epoch 6/100
13/13 [==============================] - 12s 914ms/step - loss: 1.5399 - acc: 0.3324 - val_loss: 1.5834 - val_acc: 0.3091
Epoch 7/100
13/13 [==============================] - 12s 923ms/step - loss: 1.5349 - acc: 0.3372 - val_loss: 1.5685 - val_acc: 0.3109
Epoch 8/100
13/13 [==============================] - 12s 927ms/st

#### Evaluation

In [ ]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

3/3 [==============================] - 1s 320ms/step - loss: 1.3613 - acc: 0.4182


# Training Linear Classifier with Augmentation

#### Augmentation

In [ ]:
# Configs
CROP_SIZE = 224
MIN_SCALE = 0.5
MAX_SCALE = 1.

# Experimental options
options = tf.data.Options()
options.experimental_optimization.noop_elimination = True
options.experimental_optimization.map_vectorization.enabled = True
options.experimental_optimization.apply_default_optimizations = True
options.experimental_deterministic = False
options.experimental_threading.max_intra_op_parallelism = 1

In [ ]:
@tf.function
def scale_image(image, label):
	image = tf.image.convert_image_dtype(image, tf.float32)
	return (image, label)

@tf.function
def random_apply(func, x, p):
	return tf.cond(
		tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
				tf.cast(p, tf.float32)),
		lambda: func(x),
		lambda: x)
 
@tf.function
def random_resize_crop(image, label):
  # Conditional resizing
  image = tf.image.resize(image, (260, 260))
  # Get the crop size for given min and max scale
  size = tf.random.uniform(shape=(1,), minval=MIN_SCALE*260,
		          maxval=MAX_SCALE*260, dtype=tf.float32)
  size = tf.cast(size, tf.int32)[0]
  # Get the crop from the image
  crop = tf.image.random_crop(image, (size,size,3))
  crop_resize = tf.image.resize(crop, (CROP_SIZE, CROP_SIZE))
  
  return crop_resize, label

@tf.function
def tie_together(image, label):
  # Scale the pixel values
  image, label = scale_image(image , label)
  # random horizontal flip
  image = random_apply(tf.image.random_flip_left_right, image, p=0.5)
  # Random resized crops
  image, label = random_resize_crop(image, label)
  
  return image, label

In [ ]:
trainloader = (
	train_ds
	.shuffle(1024)
	.map(tie_together, num_parallel_calls=AUTO)
	.batch(BATCH_SIZE)
	.prefetch(AUTO)
)

trainloader = trainloader.with_options(options)

#### Training

In [ ]:
# get model and compile
tf.keras.backend.clear_session()
model = get_linear_classifier(alpha=1e-6)
model.summary()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer="adam")

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train 
history = model.fit(training_ds,
                 validation_data=(testing_ds),
                 epochs=100,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


Epoch 1/100
13/13 [==============================] - 13s 1s/step - loss: 2.1446 - acc: 0.2561 - val_loss: 1.9528 - val_acc: 0.2618
Epoch 2/100
13/13 [==============================] - 12s 960ms/step - loss: 1.6818 - acc: 0.3006 - val_loss: 1.6579 - val_acc: 0.2600
Epoch 3/100
13/13 [==============================] - 12s 929ms/step - loss: 1.5806 - acc: 0.3266 - val_loss: 1.5741 - val_acc: 0.2909
Epoch 4/100
13/13 [==============================] - 11s 874ms/step - loss: 1.5441 - acc: 0.3282 - val_loss: 1.5746 - val_acc: 0.2855
Epoch 5/100
13/13 [==============================] - 12s 914ms/step - loss: 1.5295 - acc: 0.3362 - val_loss: 1.5603 - val_acc: 0.2855
Epoch 6/100
13/13 [==============================] - 12s 928ms/step - loss: 1.5194 - acc: 0.3471 - val_loss: 1.5453 - val_acc: 0.2945
Epoch 7/100
13/13 [==============================] - 12s 934ms/step - loss: 1.5120 - acc: 0.3465 - val_loss: 1.5415 - val_acc: 0.2964
Epoch 8/100
13/13 [==============================] - 12s 939ms/st

#### Evaluation

In [ ]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

3/3 [==============================] - 1s 325ms/step - loss: 1.4145 - acc: 0.3945
